In [4]:
# Randomly sample files to read
import random
from pathlib import Path

in_dir = Path("data/ladys")
sample_size = 20

wav_paths = list(in_dir.rglob("*.wav"))
if sample_size < len(wav_paths):
    sampled_paths = random.sample(wav_paths, sample_size)
else:
    sampled_paths = wav_paths  

print(len(sampled_paths))

6


In [5]:
from torchaudio.functional import resample
import torch
import torchaudio
from tqdm import tqdm

hubert = torch.hub.load(
    "bshall/hubert:main",
    f"hubert_discrete",
    trust_repo=True,
)

acoustic_units = {}
for wav_path in tqdm(sampled_paths, desc="Encoding Audio Features"):
    wav, sr = torchaudio.load(wav_path)
    wav = resample(wav, sr, 16000)
    wav = wav.unsqueeze(0)

    with torch.inference_mode():
        units = hubert.units(wav)
    acoustic_units[wav_path.stem] = units.numpy()
    

Using cache found in /home/danel/.cache/torch/hub/bshall_hubert_main
Encoding Audio Features: 100%|██████████| 6/6 [00:00<00:00, 19.47it/s]


In [6]:
for path in acoustic_units:
    print(path)
    print(acoustic_units[path])
    print()

white6
[59 35 43 50 50 51 48 20 20 20 78 78]

rose5
[97 24 13 13 13  8  8  8 85 85 85 75 73 73 29 92 30 34]

lady16
[17 17 17 95 48 48 48 48 48 20 82 82 40 57 57 57 91 91 91  5 33 78 78 74
 74]

lady2
[ 6  6 90 17 17 17 17 95 48 48 48 48 48 20 20 82 82 40 40 57 57 91 91 91
  5  5 16 74 74]

lady7
[17 17 95 95 48 48 48 48 20 82 82 82 40 57 57 57 91 91 33 33 78 78 74 74]

lady0
[ 6  6 90 17 17 17 17 95 95 48 48 48 48 91 20 33 63 82 40 40 91 91 91 91
 91 91  5]



In [7]:
# KMeans
from sklearn.cluster import KMeans

def kmeans_model(url):
    model = KMeans(100)
    checkpoint = torch.hub.load_state_dict_from_url(url)

    model.__dict__["n_features_in_"] = checkpoint["n_features_in_"]
    model.__dict__["_n_threads"] = checkpoint["_n_threads"]
    model.__dict__["cluster_centers_"] = checkpoint["cluster_centers_"].numpy()
    return model


kmeans_url = "https://github.com/bshall/dusted/releases/download/v0.1/kmeans-english-50f36a.pt"
kmeans = kmeans_model(kmeans_url)

In [9]:
from segment import segment

layer = 8
gamma = 0.05

bundle = torchaudio.pipelines.WAVLM_BASE
model = bundle.get_model()
model.eval()

encodings = {}
dusted_units = {}
for wav_path in tqdm(sampled_paths, desc="Encoding Audio Features"):
    wav, sr = torchaudio.load(wav_path)
    wav = torchaudio.functional.resample(wav, sr, 16000)

    with torch.inference_mode():
        encoding, _ = model.extract_features(wav, num_layers=layer)

    encoding = encoding[layer-1].squeeze().cpu().numpy()
    encodings[wav_path.stem] = encoding

    codes, _ = segment(encoding, kmeans.cluster_centers_, gamma)   
    dusted_units[wav_path.stem] = codes

ImportError: Numba needs NumPy 2.1 or less. Got NumPy 2.2.

In [ ]:
d_units = []
a_units = []
for path in dusted_units:
    print(path)
    print(dusted_units[path])
    d_units.append(dusted_units[path])
    print(acoustic_units[path])
    a_units.append(acoustic_units[path])
    print()
        

NameError: name 'dusted_units' is not defined

In [ ]:
import editdistance
import numpy as np

num_features = len(a_units)
dist_mat = np.zeros((num_features, num_features))

for u in range(num_features):
    for v in range(u + 1, num_features):
        dist_mat[u, v] = editdistance.eval(a_units[u], a_units[v])

print(dist_mat)

NameError: name 'a_units' is not defined